In [1]:
import torch
import numpy as np
np.random.seed(1328)
torch.random.manual_seed(1328)


In [2]:
from torch import nn
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        #Definiamo il primo livello. Dobbiamo effettuare una convoluzione 2D (ovvero su immagini)
        #Utilizziamo il modulo Conv2d che prende in input:
        # - il numero di canali in input: 1 (si tratta di immagini in scala di grigio)
        # - il numero di canali in output: 6 (le mappe di feature)
        # - la dimensione del kernel: 5 (sta per "5 X 5")
        self.C1 = nn.Conv2d(1, 6, 5)
        #Definiamo il livello di subsampling. Questo viene implementato usando il modulo "AvgPool2d"
        #Il modulo richiede in input la dimensione dei neighborhood rispetto ai quali calcolare
        # i valori medi: 2
        self.S2 = nn.AvgPool2d(2)
        #Definiamo il livello C3 in maniera analoga a quanto fatto per il livello C1:
        self.C3 = nn.Conv2d(6, 16, 5)
        #Definiamo il successivo max pooling 2d
        self.S4 = nn.AvgPool2d(2)
        #Definiamo il primo layer FC
        self.F5 = nn.Linear(256, 120)
        #Definiamo il secondo layer FC
        self.F6 = nn.Linear(120, 84)
        #Definiamo il terzo layer FC
        self.F7 = nn.Linear(84, 10)
        #Definiamo inoltre un modulo per calcolare l'attivazione ReLU
        self.activation = nn.Tanh()
    def forward(self,x):
        #Applichiamo le diverse trasformazioni in cascata
        x = self.C1(x)
        x = self.S2(x)
        x = self.activation(x) #inseriamo le attivazioni ove opportuno
        x = self.C3(x)
        x = self.S4(x)
        x = self.activation(x) #inseriamo le attivazioni ove opportuno
        x = self.F5(x.view(x.shape[0],-1)) #dobbiamo effettuare un "reshape" del tensore
        x = self.activation(x)
        x = self.F6(x)
        x = self.activation(x)
        x = self.F7(x)
        return x

In [3]:
net = LeNet()
sum([p.numel() for p in net.parameters()])

44426

In [4]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.1307,), (0.3081,))])
mnist_train = MNIST(root='mnist',train=True, download=True, transform=transform)
mnist_test = MNIST(root='mnist',train=False, download=True, transform=transform)
mnist_train_loader = DataLoader(mnist_train, batch_size=512, num_workers=2, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=512, num_workers=2)

In [5]:
from sklearn.metrics import accuracy_score
from torch.optim import SGD
from torch.autograd import Variable
def train_classification(model, lr=0.01, epochs=20, momentum=0.9, \
    train_loader=mnist_train_loader, test_loader=mnist_test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(),lr, momentum=momentum)
    loaders = {'train':train_loader, 'test':test_loader}
    losses = {'train':[], 'test':[]}
    accuracies = {'train':[], 'test':[]}
    if torch.cuda.is_available():
        model=model.cuda()
    for e in range(epochs):
        print("Primo ciclo for.")
        for mode in ['train', 'test']:
            print("Secondo ciclo for.")
            if mode=='train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0
            epoch_acc = 0
            samples = 0
            print("Mode-->",mode)
            print("Enumerate-->", loaders[mode])
            for i, batch in enumerate(loaders[mode]):
                #trasformiamo i tensori in variabili
                x=Variable(batch[0], requires_grad=(mode=='train'))
                print("x shape-->",x.shape)
                y=Variable(batch[1])
                if torch.cuda.is_available():
                    x, y = x.cuda(), y.cuda()
                    print("Con cuda")
                else:
                    print("Senza cuda")
                output = model(x)
                print(type(output))
                l = criterion(output,y)
                if mode=='train':
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                print("L-->",l.item())
                acc = accuracy_score(y.cpu().data,output.cpu().max(1)[1].data)
                epoch_loss+=l.data.item()*x.shape[0]
                epoch_acc+=acc*x.shape[0]
                samples+=x.shape[0]
                print ("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
                (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss/samples, epoch_acc/samples),
                epoch_loss/samples,
                epoch_acc/samples,
                losses[mode].append(epoch_loss))
                accuracies[mode].append(epoch_acc)
            print("Fine secondo ciclo for")
        print("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
        (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss, epoch_acc))

    print("Ho finito.")
    #restituiamo il modello e i vari log
    return model, (losses, accuracies)

In [6]:
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
cifar_train = CIFAR100(root='cifar100',train=True, download=True, transform=transform)
cifar_test = CIFAR100(root='cifar100',train=False, download=True, transform=transform)
cifar_train_loader = DataLoader(cifar_train, batch_size=512, num_workers=2, shuffle=True)
cifar_test_loader = DataLoader(cifar_test, batch_size=512, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from torch import nn
class LeNetColor(nn.Module):
    def __init__(self):
        super(LeNetColor, self).__init__()
        #ridefiniamo il modello utilizzando i moduli sequential.
        #ne definiamo due: un "feature extractor", che estrae le feature maps
        #e un "classificatore" che implementa i livelly FC
        self.feature_extractor = nn.Sequential(
        nn.Conv2d(3, 18, 5), #Input: 3 x 32 x 32. Ouput: 18 x 28 x 28
        nn.MaxPool2d(2), #Input: 18 x 28 x 28. Output: 18 x 14 x 14
        nn.ReLU(),
        nn.Conv2d(18, 28, 5), #Input 18 x 14 x 14. Output: 28 x 10 x 10
        nn.MaxPool2d(2), #Input 28 x 10 x 10. Output: 28 x 5 x 5
        nn.ReLU()
        )
        self.classifier = nn.Sequential(
        nn.Linear(700, 360), #Input: 28 * 5 * 5
        nn.ReLU(),
        nn.Linear(360, 252),
        nn.ReLU(),
        nn.Linear(252, 100)
        )
    def forward(self,x):
        #Applichiamo le diverse trasformazioni in cascata
        x = self.feature_extractor(x)
        x = self.classifier(x.view(x.shape[0],-1))
        return x

In [8]:
lenet_cifar = LeNetColor()
print(sum([p.numel() for p in lenet_cifar.parameters()]))
lenet_cifar, lenet_cifar_logs = train_classification(lenet_cifar, train_loader=cifar_train_loader, \
test_loader=cifar_test_loader, epochs=25)

382628
Primo ciclo for.
Secondo ciclo for.
Mode--> train
Enumerate--> <torch.utils.data.dataloader.DataLoader object at 0x000001C73E229400>
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.608421802520752
[train] Epoch 1/25. Iteration 0/98. Loss: 4.61. Accuracy: 0.01					 4.608421802520752 0.0078125 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.603013038635254
[train] Epoch 1/25. Iteration 1/98. Loss: 4.61. Accuracy: 0.01					 4.605717420578003 0.009765625 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.610262393951416
[train] Epoch 1/25. Iteration 2/98. Loss: 4.61. Accuracy: 0.01					 4.607232411702474 0.011067708333333334 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.608336448669434
[train] Epoch 1/25. Iteration 3/98. Loss: 4.61. Accuracy: 0.01					 4.607508420944214 0.01123046875 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'to

L--> 4.600581169128418
[train] Epoch 1/25. Iteration 39/98. Loss: 4.60. Accuracy: 0.01					 4.604664397239685 0.010498046875 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.601402282714844
[train] Epoch 1/25. Iteration 40/98. Loss: 4.60. Accuracy: 0.01					 4.604584833470787 0.010384908536585366 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.601566791534424
[train] Epoch 1/25. Iteration 41/98. Loss: 4.60. Accuracy: 0.01					 4.6045129753294445 0.01050967261904762 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.5962815284729
[train] Epoch 1/25. Iteration 42/98. Loss: 4.60. Accuracy: 0.01					 4.6043215463327805 0.010628633720930232 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.603329658508301
[train] Epoch 1/25. Iteration 43/98. Loss: 4.60. Accuracy: 0.01					 4.604299003427679 0.010564630681818182 None
x shape--> torch.Size([512, 3, 32, 32])

[train] Epoch 1/25. Iteration 78/98. Loss: 4.60. Accuracy: 0.01					 4.5997333224815655 0.013202136075949366 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.59053897857666
[train] Epoch 1/25. Iteration 79/98. Loss: 4.60. Accuracy: 0.01					 4.599618393182754 0.013330078125 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.588615894317627
[train] Epoch 1/25. Iteration 80/98. Loss: 4.60. Accuracy: 0.01					 4.599482559863432 0.013527199074074073 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.575076580047607
[train] Epoch 1/25. Iteration 81/98. Loss: 4.60. Accuracy: 0.01					 4.599184925963239 0.013814786585365854 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.585681438446045
[train] Epoch 1/25. Iteration 82/98. Loss: 4.60. Accuracy: 0.01					 4.5990222333425494 0.013883659638554216 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'tor

[test] Epoch 1/25. Iteration 19/20. Loss: 4.56. Accuracy: 0.02					 4.555313592529297 0.0234 None
Fine secondo ciclo for
[test] Epoch 1/25. Iteration 19/20. Loss: 45553.14. Accuracy: 234.00					
Primo ciclo for.
Secondo ciclo for.
Mode--> train
Enumerate--> <torch.utils.data.dataloader.DataLoader object at 0x000001C73E229400>
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.553521156311035
[train] Epoch 2/25. Iteration 0/98. Loss: 4.55. Accuracy: 0.03					 4.553521156311035 0.02734375 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.542215824127197
[train] Epoch 2/25. Iteration 1/98. Loss: 4.55. Accuracy: 0.03					 4.547868490219116 0.029296875 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.559776306152344
[train] Epoch 2/25. Iteration 2/98. Loss: 4.55. Accuracy: 0.02					 4.551837762196858 0.024739583333333332 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'

Con cuda
<class 'torch.Tensor'>
L--> 4.3328680992126465
[train] Epoch 2/25. Iteration 38/98. Loss: 4.46. Accuracy: 0.03					 4.461094782902644 0.030749198717948716 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.338600158691406
[train] Epoch 2/25. Iteration 39/98. Loss: 4.46. Accuracy: 0.03					 4.458032417297363 0.031103515625 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.290387153625488
[train] Epoch 2/25. Iteration 40/98. Loss: 4.45. Accuracy: 0.03					 4.453943508427318 0.03172637195121951 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.308584690093994
[train] Epoch 2/25. Iteration 41/98. Loss: 4.45. Accuracy: 0.03					 4.450482584181286 0.03264508928571429 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 4.294629096984863
[train] Epoch 2/25. Iteration 42/98. Loss: 4.45. Accuracy: 0.03					 4.446858084479044 0.03297601744186047 None
x shape-

...

[train] Epoch 7/25. Iteration 73/98. Loss: 3.16. Accuracy: 0.23					 3.1603532610712826 0.23434860641891891 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.130521774291992
[train] Epoch 7/25. Iteration 74/98. Loss: 3.16. Accuracy: 0.23					 3.1599555079142254 0.23411458333333332 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.1087732315063477
[train] Epoch 7/25. Iteration 75/98. Loss: 3.16. Accuracy: 0.23					 3.1592820569088587 0.23434930098684212 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.0849087238311768
[train] Epoch 7/25. Iteration 76/98. Loss: 3.16. Accuracy: 0.23					 3.1583161694662913 0.2347047483766234 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.1804051399230957
[train] Epoch 7/25. Iteration 77/98. Loss: 3.16. Accuracy: 0.23					 3.1585993613952246 0.23467548076923078 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<cla

[test] Epoch 7/25. Iteration 14/20. Loss: 3.17. Accuracy: 0.24					 3.1749849478403727 0.24453125 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.369722604751587
[test] Epoch 7/25. Iteration 15/20. Loss: 3.19. Accuracy: 0.24					 3.1871560513973236 0.241943359375 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.092660665512085
[test] Epoch 7/25. Iteration 16/20. Loss: 3.18. Accuracy: 0.24					 3.1815974992864273 0.24115349264705882 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.109515905380249
[test] Epoch 7/25. Iteration 17/20. Loss: 3.18. Accuracy: 0.24					 3.1775929662916393 0.24153645833333334 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.157850742340088
[test] Epoch 7/25. Iteration 18/20. Loss: 3.18. Accuracy: 0.24					 3.176553901873137 0.24280427631578946 None
x shape--> torch.Size([272, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--

[train] Epoch 8/25. Iteration 33/98. Loss: 3.04. Accuracy: 0.26					 3.0447818181094 0.2556295955882353 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 2.977269411087036
[train] Epoch 8/25. Iteration 34/98. Loss: 3.04. Accuracy: 0.26					 3.0428528921944755 0.25625 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.0466341972351074
[train] Epoch 8/25. Iteration 35/98. Loss: 3.04. Accuracy: 0.25					 3.0429579284456043 0.2548828125 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.0236294269561768
[train] Epoch 8/25. Iteration 36/98. Loss: 3.04. Accuracy: 0.26					 3.0424355365134574 0.2551203547297297 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.1032121181488037
[train] Epoch 8/25. Iteration 37/98. Loss: 3.04. Accuracy: 0.26					 3.0440349202407035 0.2554481907894737 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 

Con cuda
<class 'torch.Tensor'>
L--> 2.97757625579834
[train] Epoch 8/25. Iteration 73/98. Loss: 3.04. Accuracy: 0.26					 3.0432338070225073 0.2583403716216216 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 2.919248104095459
[train] Epoch 8/25. Iteration 74/98. Loss: 3.04. Accuracy: 0.26					 3.0415806643168133 0.2584375 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 2.9339258670806885
[train] Epoch 8/25. Iteration 75/98. Loss: 3.04. Accuracy: 0.26					 3.040164153826864 0.2584035773026316 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.004612922668457
[train] Epoch 8/25. Iteration 76/98. Loss: 3.04. Accuracy: 0.26					 3.039702449526106 0.25867491883116883 None
x shape--> torch.Size([512, 3, 32, 32])
Con cuda
<class 'torch.Tensor'>
L--> 3.1777267456054688
[train] Epoch 8/25. Iteration 77/98. Loss: 3.04. Accuracy: 0.26					 3.0414719917835336 0.2585136217948718 None
x shape--> tor

KeyboardInterrupt: 

In [ ]:
plot_logs_classification(lenet_cifar_logs)

In [9]:
import sys
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
